In [1]:
# Importing libraries
import pyspark
import numpy as np

In [2]:
# Creating spark enviroment
sc = pyspark.SparkContext()

22/12/21 20:44:10 WARN Utils: Your hostname, martin resolves to a loopback address: 127.0.1.1; using 192.168.0.24 instead (on interface wlp1s0)
22/12/21 20:44:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/21 20:44:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Dataset common path
common_path = "./"                      # Default dataset path
common_path = "../datasets/small_data/" # Custom dataset path

# Dataset names
phones_acc_path = common_path + "Phones_accelerometer.csv"
phones_gyr_path = common_path + "Phones_gyroscope.csv"
watch_acc_path = common_path + "Watch_accelerometer.csv"
watch_gry_path = common_path + "Watch_gyroscope.csv"

In [4]:
# Importing each dataset into a RDD
rdd_acc_phones = sc.textFile(phones_acc_path)
rdd_gyr_phones = sc.textFile(phones_gyr_path)
rdd_acc_watches = sc.textFile(watch_acc_path)
rdd_gyr_watches = sc.textFile(watch_gry_path)

In [5]:
# Defining funtion that will compute stats for each list of values
def compute_stats(values):
    # Calculating values
    mean = np.mean(values, axis=0)
    std = np.std(values, axis=0)
    max = np.max(values, axis=0)
    min = np.min(values, axis=0)

    return np.concatenate([mean, std, max, min]).ravel().tolist()

In [6]:
# Defining function to group RDD by User, Model, and class (gt) and performing required aggregates
def transform_rdd(rdd):
    # Splitting row into an array with the separator, and adding User, Model, and action (gt) as key
    rdd = rdd.map(lambda line: line.split(",")).map(lambda line: ((line[6], line[7], line[9]), (float(line[3]), float(line[4]), float(line[5]))))

    # Grouping RDD by just created key
    rdd = rdd.groupByKey().mapValues(list)

    # Calculating values
    rdd = rdd.mapValues(lambda tuple: compute_stats(tuple))

    return rdd

In [7]:
rdd_acc_phones = transform_rdd(rdd_acc_phones)
rdd_gyr_phones = transform_rdd(rdd_gyr_phones)
rdd_acc_watches = transform_rdd(rdd_acc_watches)
rdd_gyr_watches = transform_rdd(rdd_gyr_watches)

In [8]:
print(rdd_acc_phones.first())

(('a', 'nexus4', 'stand'), [-6.02649995057012, 0.9334959509016011, 8.013646013119995, 0.18455174673530322, 0.24043415892742395, 0.17599985821420452, -5.5202026, 1.9472808999999998, 8.638794, -7.0448303, -0.84251404, 7.149872])
